In [1]:
!pip install pandas


In [2]:
!pip install dython

In [5]:
import pandas as pd
import glob
import os

##  Read files

In [8]:
# Define the file path
file_path = r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/weekly_stats_demo_cleaned_for_model_v1.csv'

# Read the CSV file into a DataFrame
final_df_model = pd.read_csv(file_path)

In [10]:
# Define the file path
file_path = r'C:/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Souradeep_Trip_Info/RWRAD_all_label_wdrive.csv'

# Read the CSV file into a DataFrame
trip_df = pd.read_csv(file_path)

In [46]:
trip_df.head()

,Unnamed: 0,subj,drive,gps_lat,gps_long,time_utc,labels,week_number,day_type,year
0,10.0,RWRAD_001,4,40.779259,-96.607689,2021-03-18 18:21:48,home,11,weekday,2021
1,13.0,RWRAD_001,5,40.779266,-96.607674,2021-03-18 21:23:42,home,11,weekday,2021
2,20.0,RWRAD_001,7,40.779259,-96.607796,2021-03-18 23:58:10,home,11,weekday,2021
3,25.0,RWRAD_001,9,40.779236,-96.607628,2021-03-19 14:30:10,home,11,weekday,2021
4,31.0,RWRAD_001,12,40.779263,-96.607636,2021-03-19 15:30:53,home,11,weekday,2021


## Attch week and weekend/weekday info

In [17]:
import pandas as pd

# Convert 'time_utc' to datetime format
trip_df['time_utc'] = pd.to_datetime(trip_df['time_utc'], errors='coerce')

# Extract the week number from 'time_utc'
trip_df['week_number'] = trip_df['time_utc'].dt.isocalendar().week


In [21]:
# Weekday: Monday=0, Sunday=6
trip_df['day_type'] = trip_df['time_utc'].dt.weekday

# Convert weekday (0 to 4) to 'weekday' and (5, 6) to 'weekend'
trip_df['day_type'] = trip_df['day_type'].apply(lambda x: 'weekend' if x in [5, 6] else 'weekday')


In [44]:
# Convert the 'time_utc' column to datetime (if not already in datetime format)
trip_df['time_utc'] = pd.to_datetime(trip_df['time_utc'])

# Extract the year from 'time_utc' and create a new column 'year'
trip_df['year'] = trip_df['time_utc'].dt.year


## Extract all the required trips based on labels and day_type

In [48]:
# Define the values you're interested in
labels_of_interest = ['home', 'errand', 'medical', 'none']

# Filter rows where the 'labels' column matches any of the values in the list
filtered_df = trip_df[trip_df['labels'].isin(labels_of_interest)]


In [54]:
filtered_df.head()

,Unnamed: 0,subj,drive,gps_lat,gps_long,time_utc,labels,week_number,day_type,year
0,10.0,RWRAD_001,4,40.779259,-96.607689,2021-03-18 18:21:48,home,11,weekday,2021
1,13.0,RWRAD_001,5,40.779266,-96.607674,2021-03-18 21:23:42,home,11,weekday,2021
2,20.0,RWRAD_001,7,40.779259,-96.607796,2021-03-18 23:58:10,home,11,weekday,2021
3,25.0,RWRAD_001,9,40.779236,-96.607628,2021-03-19 14:30:10,home,11,weekday,2021
4,31.0,RWRAD_001,12,40.779263,-96.607636,2021-03-19 15:30:53,home,11,weekday,2021


In [52]:

# Step 2: Further filter based on the combinations of 'labels' and 'day_type'
filtered_df = filtered_df[
    ((filtered_df['labels'] == 'errand') & (filtered_df['day_type'] == 'weekend')) |
    ((filtered_df['labels'] == 'home') & (filtered_df['day_type'] == 'weekday')) |
    ((filtered_df['labels'] == 'medical') & (filtered_df['day_type'] == 'weekday')) |
    ((filtered_df['labels'] == 'none') & (filtered_df['day_type'] == 'weekday')) |
    ((filtered_df['labels'] == 'social') & (filtered_df['day_type'] == 'weekday'))
]

## calculate the count of trips now

In [56]:
# Define the values you're interested in for 'labels' and 'day_type'
combinations = [
    ('errand', 'weekend'),
    ('home', 'weekday'),
    ('medical', 'weekday'),
    ('none', 'weekday'),
    ('social', 'weekday')
]

# Initialize an empty list to store results
results = []

# Step 1: Loop through the combinations of 'labels' and 'day_type'
for label, day in combinations:
    # Filter the DataFrame for each combination
    filtered_combination = filtered_df[
        (filtered_df['labels'] == label) & 
        (filtered_df['day_type'] == day)
    ]
    
    # Step 2: Group by 'subj', 'year', and 'week_number' and count occurrences of 'drive' (assuming 'drive' column exists)
    count_combination = filtered_combination.groupby(
        ['subj', 'year', 'week_number']
    )['drive'].count().reset_index(name=f'{label}_{day}_count')
    
    # Append the result to the list
    results.append(count_combination)

# Step 3: Merge all the results into a single DataFrame
final_df = results[0]
for result in results[1:]:
    final_df = pd.merge(final_df, result, on=['subj', 'year', 'week_number'], how='outer')



In [60]:
final_df.head(10)

,subj,year,week_number,errand_weekend_count,home_weekday_count,medical_weekday_count,none_weekday_count,social_weekday_count
0,RWRAD_001,2021,11,1.0,6.0,1.0,NaN,NaN
1,RWRAD_001,2021,12,2.0,8.0,NaN,NaN,NaN
2,RWRAD_001,2021,14,3.0,7.0,NaN,NaN,NaN
3,RWRAD_001,2021,15,1.0,6.0,NaN,NaN,NaN
4,RWRAD_001,2021,16,1.0,7.0,1.0,6.0,NaN
5,RWRAD_001,2021,21,1.0,5.0,1.0,NaN,NaN
6,RWRAD_001,2021,23,1.0,5.0,1.0,NaN,NaN
7,RWRAD_002,2021,11,2.0,NaN,NaN,NaN,NaN
8,RWRAD_002,2021,12,1.0,8.0,1.0,1.0,NaN
9,RWRAD_002,2021,14,4.0,1.0,NaN,NaN,NaN


In [62]:
# Assuming your DataFrame is named 'df'
final_df.to_csv('end_trip_errand_info.csv', index=False)


## merge dataframes

In [65]:
# Merge the 'mean_values' DataFrame with 'final_df_model' based on 'subj', 'week_number', and 'year'
merged_df = final_df_model.merge(final_df[['subj', 'week_number', 'year', 'errand_weekend_count',	'home_weekday_count',	'medical_weekday_count',	'none_weekday_count',	'social_weekday_count']],
                                 on=['subj', 'week_number', 'year'],
                                 how='left')


In [67]:
# Assuming your DataFrame is named 'df'
merged_df.to_csv('weekly_stats_demo_cleaned_for_model_v2.csv', index=False)
